In [3]:
import pandas as pd
import numpy as np

train = pd.DataFrame()
train = pd.read_csv("train_Madison.csv", encoding = 'utf-8')
test = pd.read_csv("test_Madison.csv")


from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

C:\Users\LG\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [11]:
#train_reviews = train['text']
#test_reviews = test['text']
#all_reviews = [train_reviews, test_reviews]
#all_reviews = pd.concat(all_reviews)
x_train = train.loc[:25662,'text'].values
y_train = train.loc[:25662,'star'].values
x_test = train.loc[25663:,'text'].values
y_test = train.loc[25663:,'star'].values


tokenizer_obj = Tokenizer()
all_reviews = x_train + x_test
tokenizer_obj.fit_on_texts(all_reviews)

#pad sequence
max_length = max([len(s.split()) for s in all_reviews])

#vocab size
vocab_size = len(tokenizer_obj.word_index) + 1

x_train_tokens = tokenizer_obj.texts_to_sequences(x_train)
x_test_tokens = tokenizer_obj.texts_to_sequences(x_test)

x_train_pad = pad_sequences(x_train_tokens, maxlen= max_length, padding ='post')
x_test_pad = pad_sequences(x_test_tokens, maxlen= max_length, padding ='post')

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding



print('Build model...')

model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_DIM, input_length=max_length))
model.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Build model...


In [13]:
print('Train...')

model.fit(x_train_pad, y_train, batch_size =128, 
          epochs=25, validation_data = (x_test_pad, y_test), verbose=2)

Train...
Instructions for updating:
Use tf.cast instead.
Train on 25663 samples, validate on 25663 samples
Epoch 1/25
 - 557s - loss: -2.3503e+01 - acc: 0.0868 - val_loss: -3.6564e+01 - val_acc: 0.1014
Epoch 2/25
 - 749s - loss: -4.3420e+01 - acc: 0.0872 - val_loss: -4.3315e+01 - val_acc: 0.1014
Epoch 3/25


KeyboardInterrupt: 

In [4]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

EMBEDDING_DIM = 100
review_lines = list()
lines = train['text'].values.tolist()

for line in lines:
    tokens = word_tokenize(line)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('','',string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    review_lines.append(words)

In [5]:
len(review_lines)

51326

In [9]:
review_lines[:2]

[['see',
  'closing',
  'retool',
  'nt',
  'oppose',
  'really',
  'want',
  'see',
  'place',
  'work',
  'community',
  'got',
  'great',
  'things',
  'going',
  'historic',
  'building',
  'welltraveled',
  'street',
  'beautiful',
  'outdoor',
  'patio',
  'ample',
  'parking',
  'say',
  'visits',
  'still',
  'missing',
  'things',
  'seating',
  'guests',
  'first',
  'time',
  'went',
  'wait',
  'minutes',
  'fair',
  'place',
  'recently',
  'opened',
  'empty',
  'tables',
  'everywhere',
  'everything',
  'appeared',
  'poorly',
  'run',
  'next',
  'time',
  'made',
  'reservation',
  'still',
  'asked',
  'wait',
  'minutes',
  'table',
  'prepared',
  'following',
  'two',
  'visits',
  'great',
  'chalked',
  'early',
  'bugs',
  'working',
  'last',
  'night',
  'went',
  'seated',
  'immediately',
  'nice',
  'hostess',
  'nt',
  'tell',
  'wait',
  'staff',
  'sat',
  'minutes',
  'went',
  'back',
  'reminded',
  'finally',
  'got',
  'server',
  'cocktail',
  'se

In [10]:
import gensim

model = gensim.models.Word2Vec(sentences=review_lines, size= EMBEDDING_DIM, window= 5, workers=4, min_count=1)
words = list(model.wv.vocab)
print('Vocab size : %d' % len(words))
model.vector_size

Vocab size : 53968


100

In [23]:
model.wv.most_similar('loved')

[('enjoyed', 0.7582218647003174),
 ('fabulous', 0.7426851987838745),
 ('liked', 0.7278059720993042),
 ('fantastic', 0.7140861749649048),
 ('amazing', 0.6902319192886353),
 ('wonderful', 0.6875368356704712),
 ('loves', 0.6758674383163452),
 ('delicious', 0.6649113893508911),
 ('incredible', 0.658690333366394),
 ('phenomenal', 0.6516063213348389)]

In [20]:
model.wv.most_similar_cosmul(positive=['good', 'madison', 'food'],negative=['worst'])

[('redundantnot', 0.7373109459877014),
 ('affordable', 0.6931908130645752),
 ('inexpensive', 0.6523796916007996),
 ('town', 0.6015650629997253),
 ('pros', 0.5964080691337585),
 ('decent', 0.5838332176208496),
 ('wigerman', 0.5773665904998779),
 ('consistent', 0.5767561197280884),
 ('creative', 0.5743950009346008),
 ('spying', 0.5692256093025208)]

In [25]:
filename = 'yelp_embedding_word2vec.txt'
model.wv.save_word2vec_format(filename, binary=False)

In [37]:
import os

embeddings_index = {}
f = open(os.path.join('', 'yelp_embedding_word2vec.txt'), encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()

In [51]:
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(review_lines)
sequences = tokenizer_obj.texts_to_sequences(review_lines)

word_index = tokenizer_obj.word_index
print('Found %s unique tokens.' % len(word_index))

reviews_pad = pad_sequences(sequences, maxlen = max_length)
rating = train['star'].values
print('Shape of review tensor:', reviews_pad.shape)
print('Shape of rating tensor:', rating.shape)

Found 53968 unique tokens.
Shape of review tensor: (51326, 1251)
Shape of rating tensor: (51326,)


In [39]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [40]:
print(num_words)

53969


In [56]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding
from keras.initializers import Constant

model = Sequential()
embedding_layer = Embedding(num_words, EMBEDDING_DIM,
                            embeddings_initializer = Constant(embedding_matrix),
                            input_length = max_length,
                            trainable=False)

model.add(embedding_layer)
model.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss= 'binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [54]:
VAL_SPLIT = 0.2

indicies = np.arange(reviews_pad.shape[0])
np.random.shuffle(indicies)
reviews_pad = reviews_pad[indicies]
rating = rating[indicies]
num_validation_samples = int(VAL_SPLIT * reviews_pad.shape[0])

x_train_pad = reviews_pad[:-num_validation_samples]
y_train = rating[:-num_validation_samples]
x_test_pad = reviews_pad[-num_validation_samples:]
y_test = rating[-num_validation_samples:]

print('Shape of x_train tensor:', x_train_pad.shape)
print('Shape of y_train tensor:', y_train.shape)
print('Shape of x_test tensor:', x_test_pad.shape)
print('Shape of y_test tensor:', y_test.shape)

Shape of x_train tensor: (41061, 1251)
Shape of y_train tensor: (41061,)
Shape of x_test tensor: (10265, 1251)
Shape of y_test tensor: (10265,)


In [57]:
print('Train..')

model.fit(x_train_pad, y_train, batch_size=128, epochs=25,
          validation_data = (x_test_pad, y_test), verbose =2)

Train..
Train on 41061 samples, validate on 10265 samples
Epoch 1/25
 - 514s - loss: -3.0876e+01 - acc: 0.0935 - val_loss: -4.4097e+01 - val_acc: 0.0949
Epoch 2/25
 - 522s - loss: -4.3943e+01 - acc: 0.0941 - val_loss: -4.4097e+01 - val_acc: 0.0949
Epoch 3/25


KeyboardInterrupt: 